In [10]:
import pandas as pd

listing_id = "16069685"

sample = pd.read_csv(r"C:\Users\emman\Documents\Airbnb_Project\results\top_5_sampled_reviews.csv")
listings_df = pd.read_csv(r"C:\Users\emman\Documents\Airbnb_Project\results\Top 5 Listings.csv")

test_reviews = sample[sample['listing_id'].astype(str) == listing_id]
listing_row = listings_df[listings_df['id'].astype(str) == listing_id]

listing_info = listing_row.to_dict('records')[0]
print("Number of reviews for test:", len(test_reviews))
test_reviews.head(12)
reviews_text = "\n\n".join(test_reviews['comments'].astype(str))


Number of reviews for test: 10


In [4]:
import sys
sys.executable

'c:\\Users\\emman\\AppData\\Local\\Programs\\Python\\Python314\\python.exe'

In [12]:
import sys
!{sys.executable} -m pip install --user openai



In [20]:
def get_completion(prompt, model="gpt-4.1-nano", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = temperature
    )
    return response.choices[0].message.content

In [17]:
review_prompt = f"""
You are analysing Airbnb guest reviews for the listing with ID {listing_id}.

Based ONLY on the reviews provided, answer the following questions clearly and briefly:

1. Summarise the general feedback from guests. What does the listing do well?
2. What consistent problems or weaknesses do guests mention? Highlight anything that could be a risk for an investor.
3. What improvements could increase guest satisfaction?
4. Give an investment attractiveness score from 1–10 and explain why.

REVIEWS:
{reviews_text}

Return your answer as JSON:

{{
  "listing_id": "{listing_id}",
  "summary": "",
  "positives": [],
  "negatives": [],
  "improvements": [],
  "investment_score": "",
  "score_explanation": ""
}}
"""
response_text = get_completion(review_prompt)

output_path = rf"C:\Users\emman\Documents\Airbnb_Project\results\listing_{listing_id}_review_analysis.json"

with open(output_path, "w", encoding="utf-8") as f:
    f.write(response_text)

print("\nSaved output to:", output_path)


Saved output to: C:\Users\emman\Documents\Airbnb_Project\results\listing_16069685_review_analysis.json


The function below will put the listings data into a more structed format so the llm can read it cleaner

In [46]:
def listing_to_text(row):
    listing_text = f"""
Listing ID: {row['id']}
Name: {row['name']}
Host: {row['host_name']} (Superhost: {row['host_is_superhost']})

Location:
- Neighbourhood: {row['neighbourhood_cleansed']}
- Group: {row['neighbourhood_group_cleansed']}

Property Details:
- Property type: {row['property_type']}
- Room type: {row['room_type']}
- Accommodates: {row['accommodates']}
- Bedrooms: {row['bedrooms']}
- Beds: {row['beds']}
- Bathrooms: {row['bathrooms_text']}
- Price per night: {row['price']}

Review Metrics:
- Number of reviews: {row['number_of_reviews']}
- Review score rating: {row['review_scores_rating']}
- Reviews per month: {row['reviews_per_month']}

Neighbourhood Overview:
{row['neighborhood_overview']}
"""
    return listing_text

In [47]:
listing_row = listings_df[listings_df['id'].astype(str) == listing_id].iloc[0]

listing_text = listing_to_text(listing_row)


NameError: name 'listings_df' is not defined

In [ ]:


listing_prompt = f"""
You are analysing Airbnb LISTING METADATA for the listing with ID {listing_id}.

Based ONLY on the listing's metadata provided, answer the following questions clearly and briefly:

1. What features of this listing might make guests interested in staying here?
2. Which features are likely to be liked by guests, and which might they dislike?
3. What does the neighbourhood information suggest about convenience, demand, and guest appeal?
4. Is the property priced competitively compared to the rest of the market? Explain why or why not.
5. Based on the listing features, what risks might there be for an investor?
6. Overall, how competitive does this property appear in the local Airbnb market? (Respond in 2-4 sentences)

LISTING DATA:
{listing_text}

Return your answer as JSON ONLY:

{{
  "listing_id": "{listing_id}",
  "guest_interest_factors": [],
  "liked_features": [],
  "disliked_features": [],
  "neighbourhood_convenience_analysis": "",
  "pricing_competitiveness": "",
  "investor_risks": [],
  "market_competitiveness_summary": "",
  "final_recommendation": ""
}}
"""


response_text = get_completion(listing_prompt)
response_text
output_path = rf"C:\Users\emman\Documents\Airbnb_Project\results\listing_{listing_id}_metadata_analysis.json"

with open(output_path, "w", encoding="utf-8") as f:
    f.write(response_text)

print("\nSaved output to:", output_path)



Saved output to: C:\Users\emman\Documents\Airbnb_Project\results\listing_16069685_metadata_analysis.json


### Now we have worked on the samples lets loop through each listing to get our outputs

In [48]:
import pandas as pd
import os

# ===============================
# LOAD ALL REQUIRED DATA
# ===============================

# Paths
reviews_top_path = r"C:\Users\emman\Documents\Airbnb_Project\results\top_5_sampled_reviews.csv"
reviews_bottom_path = r"C:\Users\emman\Documents\Airbnb_Project\results\bottom_5_sampled_reviews.csv"

listings_top_path = r"C:\Users\emman\Documents\Airbnb_Project\results\Top 5 Listings.csv"
listings_bottom_path = r"C:\Users\emman\Documents\Airbnb_Project\results\bottom_5_listings.csv"

# Load CSVs
top_reviews = pd.read_csv(reviews_top_path)
bottom_reviews = pd.read_csv(reviews_bottom_path)

top_listings = pd.read_csv(listings_top_path)
bottom_listings = pd.read_csv(listings_bottom_path)

# Merge
all_reviews = pd.concat([top_reviews, bottom_reviews], ignore_index=True)
all_listings = pd.concat([top_listings, bottom_listings], ignore_index=True)

# Make sure ID fields match as strings
all_reviews['listing_id'] = all_reviews['listing_id'].astype(str)
all_listings['id'] = all_listings['id'].astype(str)

# GET ALL LISTING IDS

all_listing_ids = all_listings['id'].unique().tolist()

print("Listings to process:", all_listing_ids)


Listings to process: ['38582617', '16069685', '43723586', '16842869', '546505210695128661', '1279389366908702099', '1328494094282574396', '1354621532086945799', '1446867008477628010', '1469837763308127204']


In [49]:


results_dir = r"C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs"
os.makedirs(results_dir, exist_ok=True)

print("Saving JSON files to:", results_dir)


Saving JSON files to: C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs


## Full Prompting
 Now the prompts will loop through each listing also this includes error handling

In [ ]:
for listing_id in all_listing_ids:

    print(f"\n---- Processing Listing {listing_id} ----")

    # 1. Extract REVIEWS for this listing
    test_reviews = all_reviews[all_reviews['listing_id'] == listing_id]

    print("Number of reviews:", len(test_reviews))

    if len(test_reviews) == 0:
        print(" No reviews found for this listing. Skipping review text creation.")
        reviews_text = ""
    else:
        # Create the long block of text for prompting
        reviews_text = "\n\n".join(test_reviews['comments'].astype(str))

    # 2. Extract LISTING INFO
    listing_row = all_listings[all_listings['id'] == listing_id]

    if listing_row.empty:
        print(" No listing data found Skipping this listing.")
        continue

    # Convert table row → dictionary
    listing_info = listing_row.to_dict('records')[0]

    print("Listing info loaded successfully.")
    


    

    # REVIEW PROMPT
    review_prompt = f"""
You are analysing Airbnb guest reviews for the listing with ID {listing_id}.

Based ONLY on the reviews provided, answer the following questions clearly and briefly:

1. Summarise the general feedback from guests. What does the listing do well?
2. What consistent problems or weaknesses do guests mention? Highlight anything that could be a risk for an investor.
3. What improvements could increase guest satisfaction?
4. Give an investment attractiveness score from 1–10 and explain why.

REVIEWS:
{reviews_text}

Return your answer as JSON:

{{
  "listing_id": "{listing_id}",
  "summary": "",
  "positives": [],
  "negatives": [],
  "improvements": [],
  "investment_score": "",
  "score_explanation": ""
}}
"""

    if reviews_text.strip() != "":
        review_output = get_completion(review_prompt)
        review_save_path = os.path.join(results_dir, f"listing_{listing_id}_review_analysis.json")

        with open(review_save_path, "w", encoding="utf-8") as f:
            f.write(review_output)

        print(f"✔ Saved review analysis → {review_save_path}")
    else:
        print(" No review JSON saved (no reviews).")


    # LISTING METADATA PROMPT

    listing_prompt = f"""
You are analysing Airbnb LISTING METADATA for the listing with ID {listing_id}.

Based ONLY on the listing's metadata provided, answer the following questions clearly and briefly:

1. What features of this listing might make guests interested in staying here?
2. Which features are likely to be liked by guests, and which might they dislike?
3. What does the neighbourhood information suggest about convenience, demand, and guest appeal?
4. Is the property priced competitively compared to the rest of the market? Explain why or why not.
5. Based on the listing features, what risks might there be for an investor?
6. Overall, how competitive does this property appear in the local Airbnb market? (Respond in 2–4 sentences)

LISTING DATA:
{listing_text}

Return your answer as JSON ONLY:

{{
  "listing_id": "{listing_id}",
  "guest_interest_factors": [],
  "liked_features": [],
  "disliked_features": [],
  "neighbourhood_convenience_analysis": "",
  "pricing_competitiveness": "",
  "investor_risks": [],
  "market_competitiveness_summary": "",
  "final_recommendation": ""
}}
"""

    data_output = get_completion(listing_prompt)
    data_save_path = os.path.join(results_dir, f"listing_{listing_id}_metadata_analysis.json")

    with open(data_save_path, "w", encoding="utf-8") as f:
        f.write(data_output)

    print(f" Saved  {data_save_path}")


print("\n ALL LISTINGS COMPLETE — JSON files created for reviews + metadata!")


        



---- Processing Listing 38582617 ----
Number of reviews: 30
Listing info loaded successfully.
Prepared listing & review text for prompting.
✔ Saved review analysis → C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_38582617_review_analysis.json
 Saved  C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_38582617_metadata_analysis.json

---- Processing Listing 16069685 ----
Number of reviews: 30
Listing info loaded successfully.
Prepared listing & review text for prompting.
✔ Saved review analysis → C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_16069685_review_analysis.json
 Saved  C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_16069685_metadata_analysis.json

---- Processing Listing 43723586 ----
Number of reviews: 30
Listing info loaded successfully.
Prepared listing & review text for prompting.
✔ Saved review analysis → C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_43723586_review_ana

In [52]:
import json
import glob
import pandas as pd

top_ids = [
    "16069685",
    "16842869",
    "38582617",
    "43723586",
    "546505210695128661"
]

bottom_ids = [
    "1279389366908702099",
    "1328494094282574396",
    "1354621532086945799",
    "1446867008477628010",
    "1469837763308127204"
]

def assign_group(lid):
    if lid in top_ids:
        return "Top 5"
    elif lid in bottom_ids:
        return "Bottom 5"
    return "Unknown"


# ---------------------------------------------------------
# SAFE REVIEW FILE LOADER
# ---------------------------------------------------------

review_files = glob.glob(
    r"C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_*_review_analysis.json"
)

review_rows = []

for file in review_files:
    listing_id = file.split("listing_")[1].split("_review")[0]

    try:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

    except json.JSONDecodeError:
        # Read raw text if JSON is broken
        with open(file, "r", encoding="utf-8") as f:
            raw_text = f.read()

        data = {"raw_review_text": raw_text}

    # Assign IDs and group
    data["listing_id"] = listing_id
    data["group"] = assign_group(listing_id)

    # Convert investment_score to integer i
    if "investment_score" in data:
        try:
            data["investment_score"] = int(data["investment_score"])
        except:
            pass

    review_rows.append(data)

reviews_df = pd.DataFrame(review_rows)

reviews_df


,listing_id,summary,positives,negatives,improvements,investment_score,score_explanation,group
0,1279389366908702099,"Guests appreciate the friendly neighbors, peac...","[Friendly neighbors and peaceful neighborhood,...",[Ongoing construction and debris affecting vie...,[Address ongoing construction and improve pati...,4,While the location and neighborhood appeal are...,Bottom 5
1,1328494094282574396,Guests appreciate the excellent location near ...,[Great location near Beverly Hills and Rodeo D...,"[Poor communication from the host, especially ...",[Enhance communication and responsiveness from...,5,While the location and appearance are strong p...,Bottom 5
2,1354621532086945799,Guests have mixed experiences; some appreciate...,"[Good location and proximity to amenities, Eas...",[Misleading or inaccurate listing photos and d...,[Ensure accurate and honest listing photos and...,3,The listing shows significant issues with clea...,Bottom 5
3,1446867008477628010,"Guests appreciate the convenient location, res...",[Good location with easy access to transportat...,"[Persistent cleanliness issues, including mold...","[Thorough cleaning and mold remediation, espec...",3,The listing shows potential due to its locatio...,Bottom 5
4,1469837763308127204,"Guests find the Tiki hut charming and private,...","[Cute and quaint appearance, Quiet and private...","[Small size and challenging stairs, Separate s...","[Update and maintain cleanliness and decor, En...",4,While the property offers a charming and priva...,Bottom 5
5,16069685,"Guests highly appreciate the cozy, charming, a...","[Warm, welcoming, and friendly host Jenny, Bea...",[No significant problems mentioned in reviews],[Potentially add more detailed instructions or...,9,The listing consistently receives outstanding ...,Top 5
6,16842869,"Guests highly appreciate the peaceful, private...","[Warm, friendly, and attentive hosts, Clean, c...",[Limited indoor space (single-room layout) whi...,[Enhance outdoor amenities to encourage use in...,8,The listing demonstrates strong guest satisfac...,Top 5
7,38582617,"Guests highly appreciate the cozy, stylish, an...",[Excellent host responsiveness and hospitality...,[No significant negatives mentioned in reviews],[Potentially enhance outdoor space or add more...,9,The listing demonstrates high guest satisfacti...,Top 5
8,43723586,Guests highly praise the host's responsiveness...,[Excellent host responsiveness and communicati...,"[Lofted area has short ceilings, which may be ...","[Enhance outdoor lighting for safety at night,...",8,The listing demonstrates strong guest satisfac...,Top 5
9,546505210695128661,"Guests highly appreciate the unique, charming,...","[Unique and charming decor with vibrant, eclec...",[No significant weaknesses or recurring issues...,[Potentially adding more detailed instructions...,8,The listing demonstrates strong guest satisfac...,Top 5


In [53]:


# Directory containing  JSON files
metadata_files = glob.glob(
    r"C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs\listing_*_metadata_analysis.json"
)

rows_meta = []

for file in metadata_files:
    listing_id = file.split("listing_")[1].split("_metadata")[0]

    with open(file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print(" JSON formatting error in:", file)
            continue

    # add listing_id 
    data["listing_id"] = listing_id
    rows_meta.append(data)

# Convert to DataFrame
metadata_df = pd.DataFrame(rows_meta)

print("Loaded metadata summaries:")
metadata_df.head()
metadata_df["group"] = metadata_df["listing_id"].apply(
    lambda x: "Top 5" if x in top_ids else "Bottom 5"
)
metadata_comparison = metadata_df[
    [
        "listing_id",
        "group",
        "guest_interest_factors",
        "liked_features",
        "disliked_features",
        "neighbourhood_convenience_analysis",
        "pricing_competitiveness",
        "investor_risks",
        "market_competitiveness_summary",
        "final_recommendation"
    ]
]

metadata_comparison


Loaded metadata summaries:


,listing_id,group,guest_interest_factors,liked_features,disliked_features,neighbourhood_convenience_analysis,pricing_competitiveness,investor_risks,market_competitiveness_summary,final_recommendation
0,1279389366908702099,Bottom 5,[Proximity to attractions (10-minute walk to e...,"[Location convenience, Private entire unit, Lo...","[Low review score (3.4), Limited amenities (no...","Located in Marina del Rey, a desirable area wi...",The price appears competitive given the locati...,[Low review score indicating potential guest d...,The property offers a competitive price in a p...,Consider this listing as a budget-friendly opt...
1,1328494094282574396,Bottom 5,[Proximity to attractions (10-minute walk to e...,"[Private entire rental unit, Central location ...","[Low review score (3.4), Limited amenities (no...",The location in Marina del Rey suggests high c...,The price of $83 appears competitive given the...,[Low review score indicating potential guest d...,This property offers a competitive price in a ...,The listing has potential due to its prime loc...
2,1354621532086945799,Bottom 5,[Proximity to attractions (10-minute walk to e...,"[Location convenience, Private space, Affordab...","[Low review score (3.4), Limited amenities (no...","Located in Marina del Rey, a desirable area wi...",The price appears competitive given the locati...,[Low review score indicating potential quality...,The property offers a competitive price in a p...,Consider investing if improvements are made to...
3,1446867008477628010,Bottom 5,[Proximity to attractions (10-minute walk to e...,"[Entire rental unit suitable for 2 guests, Pri...","[No bedrooms, only beds (may be less private o...","Located in Marina del Rey, the area offers hig...",The $83 nightly rate appears competitive given...,[Low review score indicating potential guest d...,The property is competitively priced and well-...,Consider investing if improvements are made to...
4,1469837763308127204,Bottom 5,[Proximity to attractions (10-minute walk to e...,"[Entire rental unit with private bathroom, Cen...",[Low review score (3.4) indicating possible gu...,The Marina del Rey location offers high conven...,The price of $83 is competitive given the cent...,[Low review score indicating potential guest d...,This property offers a competitive price in a ...,Consider monitoring guest feedback and reviews...
5,16069685,Top 5,[Proximity to local attractions (10-minute wal...,"[Location close to amenities, Private entire u...",[Low review score (3.4) indicating possible is...,"Located in Marina del Rey, a desirable area wi...",The price of $83 is likely competitive given t...,[Low review score may indicate inconsistent gu...,This property offers a competitive price in a ...,Consider if the focus is on budget-conscious t...
6,16842869,Top 5,[Proximity to attractions (10-minute walk to e...,"[Location convenience, Private entire unit, Af...","[Low review score (3.4), Limited amenities (no...","Located in Marina del Rey, a desirable area wi...",The price of $83 is competitive given the loca...,[Low review score indicating potential guest d...,This property offers a competitive price point...,Consider if the price and location outweigh co...
7,38582617,Top 5,[Proximity to attractions (10-minute walk to e...,"[Entire rental unit suitable for 2 guests, Pri...",[Low review score (3.4) indicating possible is...,"Located in Marina del Rey, a desirable area wi...","The price of $83 is competitive for the area, ...",[Low review score may indicate inconsistent gu...,The property offers a competitive price in a p...,Consider for budget-conscious travelers seekin...
8,43723586,Top 5,[Proximity to local attractions (10-minute wal...,"[Private entire home/apt, Good location in Mar...","[Low review score (3.4/5), Limited amenities (...",The neighbourhood offers high convenience with...,The price appears competitive given the locati...,[Low review score indicating potential guest d...,This property offers a competitiv

In [51]:
all_reviews.to_csv("all_reviews_export.csv", index=False)
reviews_df.to_csv("reviews_df_export.csv", index=False)
top_reviews.to_csv("top_reviews_export.csv", index=False)
bottom_reviews.to_csv("bottom_reviews_export.csv", index=False)
